In [3]:
!pip install opencv-python

  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)


In [4]:
!pip install keras

In [5]:
!pip install tensorflow

  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0


ERROR: textract 1.6.4 has requirement six~=1.12.0, but you'll have six 1.15.0 which is incompatible.
ERROR: mlxtend 0.20.0 has requirement scikit-learn>=1.0.2, but you'll have scikit-learn 1.0.1 which is incompatible.


In [6]:
import os
import cv2 # pip install opencv-python
import  pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras 
from glob import glob
from keras.models import Sequential
from keras.layers import BatchNormalization,MaxPool2D,Dense,Conv2D,Flatten
from keras.callbacks import EarlyStopping,LearningRateScheduler
from sklearn.svm import SVC
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from tqdm import tqdm
from PIL import Image
warnings.filterwarnings('ignore')

**Tensor Flow**

**1. Import the data**

In [7]:
vehicle_dir = 'data/vehicles/'
non_vehicle_dir = 'data/non-vehicles/'

In [8]:
vehicle_names = os.listdir(vehicle_dir)
non_vehicle_names = os.listdir(non_vehicle_dir)

**2. Explore the data**

In [9]:
print("No of vehicle images:", len(vehicle_names))
print("No of non-vehicle images:", len(non_vehicle_names))

No of vehicle images: 8792
No of non-vehicle images: 8968


**3. Prepare the Data**

In [10]:
cat = {
    'non-vehicles': 0,
    'vehicles': 1
}

# get all the paths for vehicles and non-vehicles
def get_paths(folder):
    
    path = []
    labels = []
    for dirname, _, filenames in os.walk(folder):
        dir_class = dirname.split('/')[1] # either vehicles or non-vehicles
        for filename in filenames:
            path.append(os.path.join(dirname, filename))
            labels.append(cat[dir_class])
    
    return pd.DataFrame({'path': path, 'class': labels})

In [11]:
vehicles_df = get_paths(vehicle_dir)
vehicles_df.head()

,path,class
0,data/vehicles/1.png,1
1,data/vehicles/10.png,1
2,data/vehicles/1000.png,1
3,data/vehicles/1001.png,1
4,data/vehicles/1002.png,1


In [12]:
non_vehicles_df = get_paths(non_vehicle_dir)
non_vehicles_df.head()

,path,class
0,data/non-vehicles/extra1.png,0
1,data/non-vehicles/extra10.png,0
2,data/non-vehicles/extra100.png,0
3,data/non-vehicles/extra1000.png,0
4,data/non-vehicles/extra1001.png,0


In [13]:
df = pd.concat([vehicles_df, non_vehicles_df])
df.shape

(17760, 2)

In [14]:
df.head()

,path,class
0,data/vehicles/1.png,1
1,data/vehicles/10.png,1
2,data/vehicles/1000.png,1
3,data/vehicles/1001.png,1
4,data/vehicles/1002.png,1


In [15]:
# shuffle dataset using sample
df = df.sample(frac=1, random_state=1).reset_index(drop=True)
df.head()

,path,class
0,data/vehicles/3519.png,1
1,data/non-vehicles/image604.png,0
2,data/vehicles/5546.png,1
3,data/non-vehicles/image525.png,0
4,data/vehicles/far (574).png,1


In [16]:
# we sampled the first 200 rows to start out
sample_df = df.iloc[:, :] # if you want to sample just the first 200 rows do -> sample_df = df.iloc[:200, :]
sample_df

,path,class
0,data/vehicles/3519.png,1
1,data/non-vehicles/image604.png,0
2,data/vehicles/5546.png,1
3,data/non-vehicles/image525.png,0
4,data/vehicles/far (574).png,1
...,...,...
17755,data/non-vehicles/extra334.png,0
17756,data/non-vehicles/image575.png,0
17757,data/vehicles/5675.png,1
17758,data/non-vehicles/extra4531.png,0


In [17]:
sample_df['class'].value_counts() 

0    8968
1    8792
Name: class, dtype: int64

In [18]:
def load_images(paths, img_size):
    images = []
    
    for path in paths:
        img = cv2.imread(path)
        img = cv2.resize(img, img_size) #resize the images
        img = np.array(img)
        
        images.append(img)
    
    images = np.array(images)
    images = images.astype(np.int64)
    
    #scale images
#     images = images/255
    return images

In [19]:
images = load_images(sample_df['path'], (64, 64))
print(images.shape)

(17760, 64, 64, 3)


In [20]:
x = images
y = sample_df["class"]

#train test split - 75% train, 25% test

x_training, x_test = x[:13320,:], x[13320:,:]
y_training, y_test = y[:13320], y[13320:]

In [21]:
x_training.shape

(13320, 64, 64, 3)

In [22]:
x_test.shape

(4440, 64, 64, 3)

In [23]:
y_training.shape

(13320,)

In [24]:
y_test.shape

(4440,)

In [25]:
# increases accuracy (even when not using loss function) 
tf.cast(x_training, tf.int64)
tf.cast(y_training, tf.int64)

<tf.Tensor: shape=(13320,), dtype=int64, numpy=array([1, 0, 1, ..., 0, 1, 1], dtype=int64)>

**4. Build the Model**

In [26]:
# TensorFlow and tf.keras
import tensorflow as tf

print(tf.__version__)
vehicle_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(64, 64, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3)
])

2.6.0


In [23]:
# vehicle_model = tf.keras.Sequential([
#     tf.keras.layers.Flatten(input_shape=(64, 64, 3)),
#     tf.keras.layers.Dense(16, activation='sigmoid', # first hidden layer has 16 inputs, each nodes activation will be a sigmoid function 
#                           name='fc1', input_shape=(64, 64, 3)), 
#     tf.keras.layers.Dense(3, name='fc2', activation='relu')])

In [24]:
# vehicle_model.summary()

**5. Train the Model & 6. Evaluate the Model**

In [27]:
vehicle_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [28]:
vehicle_model.fit(x_training, y_training, epochs=50)

Epoch 1/50
417/417 [==============================] - 6s 11ms/step - loss: 54.3669 - accuracy: 0.8135
Epoch 2/50
417/417 [==============================] - 5s 11ms/step - loss: 12.1726 - accuracy: 0.8786: 0s - loss: 13.1698 - acc
Epoch 3/50
417/417 [==============================] - 5s 11ms/step - loss: 5.4757 - accuracy: 0.8900 0s - loss: 5.4867 - accuracy: 0.88
Epoch 4/50
417/417 [==============================] - 5s 11ms/step - loss: 1.9251 - accuracy: 0.9086
Epoch 5/50
417/417 [==============================] - 5s 11ms/step - loss: 0.4845 - accuracy: 0.9122
Epoch 6/50
417/417 [==============================] - 5s 11ms/step - loss: 0.2908 - accuracy: 0.9029
Epoch 7/50
417/417 [==============================] - 5s 11ms/step - loss: 0.2597 - accuracy: 0.8962
Epoch 8/50
417/417 [==============================] - 5s 11ms/step - loss: 0.2508 - accuracy: 0.9098
Epoch 9/50
417/417 [==============================] - 5s 11ms/step - loss: 0.1980 - accuracy: 0.9286 0s - loss: 0.1986 - accu
Epo

In [27]:
# vehicle_model.compile(optimizer='sgd', 
#               loss="mse",
#               metrics=['mae', 'mse', 'acc'])
# vehicle_model.fit(x_training, y_training, 
#                   epochs = 100, 
#                   batch_size = 5,
#                   verbose = 1)

**7. Make Predictions**

In [1]:
y_pred = vehicle_model.predict(x_test)
y_pred
# this crashes the kernel because our sample size is so large when not trying to use all images we got an accuracy of 97.5%

NameError: name 'vehicle_model' is not defined

In [29]:
test_loss = vehicle_model.evaluate(x,  y, verbose=2)

555/555 - 4s - loss: 0.1654 - accuracy: 0.9473


In [30]:
probability_model = tf.keras.Sequential([vehicle_model, 
                                         tf.keras.layers.Softmax()])

In [31]:
predictions = probability_model.predict(x_test)

In [32]:
# Example: the first prediction. 
predictions[0]

array([9.279019e-01, 7.202491e-02, 7.326257e-05], dtype=float32)

Predictions returns a number for each class which represents the probability of it being in that class. 

**SVM**

In [33]:
#SVM takes 2D input to train
X_train = np.array(x_training).reshape(len(x_training), -1)
X_train.shape

(13320, 12288)

In [34]:
y_training.shape

(13320,)

In [35]:
X_test = np.array(x_test).reshape(len(x_test), -1)
X_test.shape

(4440, 12288)

In [36]:
# SVM model
svc = SVC(probability=True)
svc.fit(X_train, y_training)
y_pred2 = svc.predict(X_test)

In [37]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred2))

0.977027027027027


In [ ]:
#figure out how many images we want to process for predictions so kernel doesn't crash, model works
#accuracy and loss function (what do we mean by loss function and can we still cast?)
#plot a few images
#add more observations and headers that match the assignment outline delete commented code (how do we interpret the predictions?)
#cite sources (like which parts are based off of his code)
#we can make our code even more exactly like the process he used (I'm fine not doing that tho) and 

#DONE - changed sample size to the entire batch instead of a small sample (it's still 75% training) 
#DONE - standardization --> resizing of image (32, 32) (64 by 64 should be good, it's more detailed). 
    # we also don't need to divide by 266 because it's not a monochrome image
#DONE - keras model accuracy - 0.9
    # can mess with batch size (didn't need to)
    # changed to relu activation
    # increased epochs to 50 instead of 10
    
#DONE - svm model accuracy - 0.98